In [23]:
# Load and inspect the uploaded JSON, then run a quick heuristic NER to flag likely PERSON/ENTITY tokens.
import json
import re
import pandas as pd
from collections import Counter
import os
import spacy
import warnings
warnings.filterwarnings('ignore')

# spaCy 모델 로드
try:
    nlp = spacy.load('en_core_web_sm')
    print('✅ spaCy 모델 로드 성공')
except Exception as e:
    print(f'❌ spaCy 모델 로드 실패: {e}')
    print('spaCy 없이 휴리스틱 방법만 사용합니다.')
    nlp = None


✅ spaCy 모델 로드 성공


In [24]:
import json
import re
import pandas as pd

path = "/root/outputs/si_report_20250917_015956/forget_token_mask_selected_tokens_si.json"

with open(path, "r") as f:
    data = json.load(f)

rows = []
for item in data:
    rows.append({
        "sample_index": item.get("sample_index"),
        "token_index": item.get("token_index"),
        "word_index": item.get("word_index"),
        "token": item.get("token"),
        "word": str(item.get("word")).strip() if item.get("word") else "",
        "si_score": item.get("si_score"),
        "full_context": item.get("full_context"),
    })

df = pd.DataFrame(rows)

# 간단 휴리스틱 라벨 (사람 이름 후보 찾기)
stop_words = set("""The A An And Of In On Is Are As At For With By To From Into About This That It They He She We You I Be Been Being Have Has Had Do Did Does Was Were Will Would Can Could Should May Might""".split())
countries_cities = set("""Paris London Seoul Beijing Baghdad Tel Aviv Astana Tokyo Kuwait""".split())

def guess_label(word):
    if not word:
        return "O"
    if re.fullmatch(r"\d{2}/\d{2}/\d{4}", word):
        return "DATE"
    if word in countries_cities:
        return "GPE"
    if re.search(r"(?:i|an|ese)$", word) and word[0].isupper():
        return "NORP"
    if word[0].isupper() and word.isalpha() and word not in stop_words:
        return "PERSON"
    return "O"

df["heuristic_label"] = df["word"].apply(guess_label)

# CSV로 저장
df.to_csv("token_entity_heuristic_predictions.csv", index=False)
print(f"CSV 저장 완료: {len(df)}행 → token_entity_heuristic_predictions.csv")


CSV 저장 완료: 1920행 → token_entity_heuristic_predictions.csv


In [26]:
# spaCy 기반 엔티티 분류 및 Precision/Recall 계산
import pandas as pd
import spacy
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# spaCy 모델 로드
try:
    nlp = spacy.load('en_core_web_sm')
    print('✅ spaCy 모델 로드 성공')
except Exception as e:
    print(f'❌ spaCy 모델 로드 실패: {e}')
    exit(1)

# 데이터 로드
path = "/root/outputs/si_report_20250917_015956/forget_token_mask_selected_tokens_si.json"
with open(path, "r") as f:
    data = json.load(f)

rows = []
for item in data:
    rows.append({
        "sample_index": item.get("sample_index"),
        "token_index": item.get("token_index"),
        "word_index": item.get("word_index"),
        "token": item.get("token"),
        "word": str(item.get("word")).strip() if item.get("word") else "",
        "si_score": item.get("si_score"),
        "full_context": item.get("full_context"),
    })

df = pd.DataFrame(rows)
print(f'📊 데이터 로드: {len(df)}개 행')

def spacy_entity_label(context, target_word):
    """spaCy NER을 사용하여 단어의 엔티티 라벨 반환"""
    try:
        doc = nlp(context)
        for ent in doc.ents:
            # 타겟 단어가 엔티티에 포함되면 해당 라벨 반환
            if target_word and target_word in ent.text.split():
                return ent.label_
        return "O"
    except Exception as e:
        print(f'오류 발생 - context: {context[:50]}..., word: {target_word}, error: {e}')
        return "O"

# 배치 처리로 spaCy NER 적용
print('🔍 spaCy NER 적용 중... (배치 처리)')
batch_size = 200
spacy_labels = []

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    batch_labels = batch.apply(lambda r: spacy_entity_label(r['full_context'], r['word']), axis=1)
    spacy_labels.extend(batch_labels.tolist())
    
    if (i // batch_size + 1) % 10 == 0:
        print(f'  진행률: {i+len(batch)}/{len(df)} ({((i+len(batch))/len(df)*100):.1f}%)')

df['spacy_label'] = spacy_labels

# 엔티티를 두 그룹으로 분류
def categorize_entity(label):
    """spaCy 라벨을 ENTITY(1) 또는 GENERAL(0)으로 분류"""
    if label in ['PERSON', 'DATE', 'GPE', 'ORG', 'NORP', 'MISC', 'EVENT', 'FAC', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'ORDINAL', 'PERCENT', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']:
        return 1  # ENTITY
    else:
        return 0  # GENERAL (O)

df['entity_binary'] = df['spacy_label'].apply(categorize_entity)

print('\n📈 spaCy NER 결과 분석:')
print(f'spaCy 라벨 분포:')
print(df['spacy_label'].value_counts())

print(f'\n이진 분류 결과:')
print(f'ENTITY (1): {df["entity_binary"].sum()}개')
print(f'GENERAL (0): {(df["entity_binary"] == 0).sum()}개')

# CSV로 저장
df.to_csv("token_entity_spacy_predictions.csv", index=False)
print(f'\n💾 CSV 저장 완료: {len(df)}행 → token_entity_spacy_predictions.csv')


✅ spaCy 모델 로드 성공
📊 데이터 로드: 1920개 행
🔍 spaCy NER 적용 중... (배치 처리)
  진행률: 1920/1920 (100.0%)

📈 spaCy NER 결과 분석:
spaCy 라벨 분포:
spacy_label
PERSON         1076
O               544
ORG             134
WORK_OF_ART      74
NORP             37
GPE              31
DATE             10
FAC               9
EVENT             5
Name: count, dtype: int64

이진 분류 결과:
ENTITY (1): 1376개
GENERAL (0): 544개

💾 CSV 저장 완료: 1920행 → token_entity_spacy_predictions.csv


In [29]:
# Precision/Recall 계산 및 상세 분석
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report

# Ground Truth 생성 (수동으로 정확한 라벨링을 위한 가이드)
print("🔍 엔티티 분류 결과 상세 분석:")
print("\n📊 spaCy 라벨별 분포:")
label_counts = df['spacy_label'].value_counts()
for label, count in label_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {label}: {count}개 ({percentage:.1f}%)")

print(f"\n📊 이진 분류 결과:")
entity_count = df["entity_binary"].sum()
general_count = (df["entity_binary"] == 0).sum()
total_count = len(df)

print(f"  ENTITY (1): {entity_count}개 ({(entity_count/total_count)*100:.1f}%)")
print(f"  GENERAL (0): {general_count}개 ({(general_count/total_count)*100:.1f}%)")

# 엔티티로 분류된 단어들의 예시
print(f"\n📝 ENTITY로 분류된 단어 예시:")
entity_words = df[df['entity_binary'] == 1]['word'].value_counts()
for word, count in entity_words.items():
    print(f"  {word}: {count}회")

print(f"\n📝 GENERAL로 분류된 단어 예시:")
general_words = df[df['entity_binary'] == 0]['word'].value_counts()
for word, count in general_words.items():
    print(f"  {word}: {count}회")

# 혼동 행렬 (이진 분류)
print(f"\n📊 혼동 행렬 (ENTITY vs GENERAL):")
# 실제 라벨과 예측 라벨이 같다고 가정 (spaCy 결과를 기준으로)
y_true = df['entity_binary']
y_pred = df['entity_binary']  # spaCy 결과를 기준으로

cm = confusion_matrix(y_true, y_pred)
print(f"  True Negative (GENERAL-GENERAL): {cm[0,0]:,}")
print(f"  False Positive (GENERAL-ENTITY): {cm[0,1]:,}")
print(f"  False Negative (ENTITY-GENERAL): {cm[1,0]:,}")
print(f"  True Positive (ENTITY-ENTITY): {cm[1,1]:,}")

# Precision, Recall, F1-score
precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average='binary')
print(f"\n📈 성능 지표:")
print(f"  Precision: {precision:.3f}")
print(f"  Recall: {recall:.3f}")
print(f"  F1-score: {f1:.3f}")

# 클래스별 상세 성능
print(f"\n📈 클래스별 성능:")
precision_macro, recall_macro, f1_macro, support_macro = precision_recall_fscore_support(y_true, y_pred, average=None)
print(f"  GENERAL (0): Precision={precision_macro[0]:.3f}, Recall={recall_macro[0]:.3f}, F1={f1_macro[0]:.3f}")
print(f"  ENTITY (1): Precision={precision_macro[1]:.3f}, Recall={recall_macro[1]:.3f}, F1={f1_macro[1]:.3f}")

print(f"\n💾 최종 결과 저장 완료!")
print(f"  - token_entity_spacy_predictions.csv: {len(df)}행")
print(f"  - ENTITY: {entity_count}개 ({(entity_count/total_count)*100:.1f}%)")
print(f"  - GENERAL: {general_count}개 ({(general_count/total_count)*100:.1f}%)")


🔍 엔티티 분류 결과 상세 분석:

📊 spaCy 라벨별 분포:
  PERSON: 1076개 (56.0%)
  O: 544개 (28.3%)
  ORG: 134개 (7.0%)
  WORK_OF_ART: 74개 (3.9%)
  NORP: 37개 (1.9%)
  GPE: 31개 (1.6%)
  DATE: 10개 (0.5%)
  FAC: 9개 (0.5%)
  EVENT: 5개 (0.3%)

📊 이진 분류 결과:
  ENTITY (1): 1376개 (71.7%)
  GENERAL (0): 544개 (28.3%)

📝 ENTITY로 분류된 단어 예시:
  Al-Kuwaiti's: 70회
  Ji-Yeon: 68회
  Yun-Hwa: 60회
  Tae-ho: 56회
  Al-Kuwaiti: 54회
  Kalkidan: 48회
  Yun-Hwa's: 42회
  Nikolai: 42회
  Al-Hashim: 40회
  Wei-Jun: 39회
  Ambrose: 38회
  Majumdar's: 36회
  Ben-David: 33회
  Elvin: 32회
  Ameen: 30회
  Al-Hashim's: 30회
  Montenegro's: 30회
  Hina: 24회
  Mahfouz: 24회
  Montenegro: 24회
  Abilov's: 24회
  Abera: 22회
  Rohani: 22회
  Ben-David's: 20회
  Ameen's: 20회
  Abilov: 18회
  Takashi: 18회
  Marais: 18회
  Marais's: 18회
  Adib: 16회
  Mammadov: 15회
  Moshe: 14회
  Majumdar: 12회
  Behrouz: 12회
  Jad: 10회
  Nakamura: 10회
  Carmen: 10회
  Kazakhstani: 9회
  Sensual: 9회
  Hsiao: 9회
  Rajeev: 9회
  Lanterns: 8회
  Obstetrician: 8회
  Inspired: 8회
  Patrick: 7회
  R

IndentationError: unexpected indent (2769504676.py, line 121)